### The Leading Edge - Geophysical Tutorial (TLE April 2017)
# Having fun with electromagnetic modeling

Dieter Werthmüller

*This Jupyter Notebook accompanies the Geophysical Tutorial from The Leading Edge April 2017 edition, entitled *<**TODO: Enter final title**>*. You can play with all of the examples in this tutorial yourself in the notebook and data at https://github.com/seg. Please refer to the tutorial in the February 2016 issue for help getting started.*


## Electromagnetic modelling

Why 1D modelling (speed, isolated effects);
Describe existing, available codes with their advantages and limitations


### `empymod`
Describe the code; fht/qwe and fht/qwe/fftlog; arbitrary rotade source and receiver with length and source moment; low to high frequencies $\rho_h, \lambda, \mu_h, \mu_v, \epsilon_h, \epsilon_v$

There are many ways to get `empymod`. You can install the latest stable version with `conda`

    conda install -c prisae empymod
    
or with `pip`

    pip install empymod
    
Alternatively, you can get the code from https://github.com/prisae/empymod and install it via

    python setup.py install

or simply put it in your working directory or Python-PATH to use it without installation at all.

The documentation is hostet on http://empymod.readthedocs.io.

Minimum requirement is Python 3.4 or higher, and `NumPy` and `SciPy` (I recommend the [Anaconda Python Distribution](https://www.continuum.io/downloads)).

*Last but not least, if you are interested in this please get involved! I am open for suggestions and pull requests, thankful for bug reports, and happy if I see that you fork it and create something awesome of your own.*


Notes: The previous 19 Tutorials had (in the article, not the notebook):
- 0-33 lines of code
- 2-4 pages
- 0-4 equations
- 1-6 figures
- 0-3 tables

Matt suggests ~1500 words

In [8]:
# NumPy for array-creation
import numpy as np

# Plot settings
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# empymod
from empymod import bipole

## Define the standard model

As e.g. in Key, 2009

## 1. Bipole versus dipole

## 2. Amplitude and phase of entire xy-plane

E.g. like in Hunziker et al, 2015

## 3. NAR

E.g. Hesthammer and Stefatos, 2010, figure 1

Offset vs normalized amplitud: NMVO (normalized magnitude vs offset); NAR: Normalised anomalous amplitude; for the fundamental frequency

## 4. Frequency-offset crossplot

## 5. Frequency versus time

Rule of thumb: Offset > 5 times the bipole length = > model as dipole; Why bother? Much faster

## 6. Anisotropy
Pitfall: ...with anisotropy x... => what anisotropy?

What type of anisotropy?
$$
\lambda = \sqrt{\frac{\rho_v}{\rho_h}}\ , \qquad \text{and} \qquad \rho_m = \sqrt{\rho_v\rho_h}
$$

$$
\lambda\rho_h = \rho_m = \frac{1}{\lambda}\rho_v
$$

Werthmüller, 2009 and presented in e.g. Hobbs et al, 2009

## 7. Common-source gather

E.g. Ziolkowski et al, 2011, figures 9 and 10.


## References

**Anderson, W. L., 1979**, Numerical integration of related Hankel transforms of orders 0 and 1 by adaptive digital filtering: Geophysics, 44, 1287-1305.  
DOI: [10.1190/1.1441007](http://dx.doi.org/10.1190/1.1441007)

**Hamilton, A. J. S., 2000**, Uncorrelated modes of the non-linear power spectrum: Monthly Notices of the Royal Astronomical Society, 312, pages 257-284.  
DOI: [10.1046/j.1365-8711.2000.03071.x](http://dx.doi.org/10.1046/j.1365-8711.2000.03071.x)  
Software: [casa.colorado.edu/~ajsh/FFTLog](http://casa.colorado.edu/~ajsh/FFTLog)

**Hesthammer, J., and A. Stefatos, 2010** ,The performance of CSEM as a de‐risking tool in oil and gas exploration: SEG Technical Program Expanded Abstracts, 675-679.  
DOI: [10.1190/1.3513873](http://dx.doi.org/10.1190/1.3513873)


**Hobbs, B. A., D. Werthmüller, and F. Engelmark, 2009**, Apparent anisotropy derived from transient electromagnetic earth responses: SEG Technical Program Expanded Abstracts, 744-748.  
DOI: [10.1190/1.3255861](http://dx.doi.org/10.1190/1.3255861)

**Hunziker, J., J. Thorbecke, and E. Slob, 2015**, The electromagnetic response in a layered vertical transverse isotropic medium: A new look at an old problem: Geophysics, 80, F1-F18.  
DOI: [10.1190/geo2013-0411.1](http://dx.doi.org/10.1190/geo2013-0411.1)  
Software: [software.seg.org/2015/0001](http://software.seg.org/2015/0001)
   
**Key, K., 2009**, 1D inversion of multicomponent, multifrequency marine CSEM data: Methodology and synthetic studies for resolving thin resistive layers: Geophysics, 74, F9-F20.  
DOI: [10.1190/1.3058434](http://dx.doi.org/10.1190/1.3058434)  
Software: [marineemlab.ucsd.edu/Projects/Occam/1DCSEM](http://marineemlab.ucsd.edu/Projects/Occam/1DCSEM)

**Key, K., and J. Ovall, 2011**, A parallel goal-oriented adaptive finite element method for 2.5-D electromagnetic modelling: Geophysical Journal International, 186, 137-154.  
DOI: [10.1111/j.1365-246X.2011.05025.x](http://dx.doi.org/10.1111/j.1365-246X.2011.05025.x)  
Software: [mare2dem.ucsd.edu](http://mare2dem.ucsd.edu)

**Key, K., 2012**, Is the fast Hankel transform faster than quadrature?: Geophysics, 77, F21-F30.  
DOI: [10.1190/GEO2011-0237.1](http://dx.doi.org/10.1190/GEO2011-0237.1)  
Software: [software.seg.org/2012/0003](http://software.seg.org/2012/0003)

**Werthmüller, D., 2009**, Inversion of Multi-Transient EM data from anisotropic media: M.Sc. Thesis.  
UUID: [f4b071c1-8e55-4ec5-86c6-a2d54c3eda5a](http://repository.tudelft.nl/view/ir/uuid:f4b071c1-8e55-4ec5-86c6-a2d54c3eda5a)

**Ziolkowski, A., D. Wright, and J. Mattsson, 2011**, Comparison of pseudo-random binary sequence and square-wave transient controlled-source electromagnetic data over the Peon gas discovery, Norway: Geophysical Prospecting, 59, 1114-1131.  
DOI: [10.1111/j.1365-2478.2011.01006.x](http://dx.doi.org/10.1111/j.1365-2478.2011.01006.x)
